In [1]:
import numpy as np
import pandas as pd
from pymongo import MongoClient
import mysql.connector
from datetime import datetime
import requests
import json

# Data Analysis Using MongoDB

## Connect to Database

In [2]:
# # MongoDB setup
transaction = MongoClient('mongodb://localhost:27017/')

# Connect to your database
db = transaction['Coinut']

# MySQL setup
mysql_db = mysql.connector.connect(
  user='root', password='liuwinq1', host='localhost', database='coinut')
mysql_cursor = mysql_db.cursor()

## Fetch Data

In [3]:
# Select the collection from the database
collection = db['transaction']

# Fetch all doc data
data1 = collection.find()

In [4]:
# Count the number of documents
num_transactions = collection.count_documents({})
print(f"Number of transactions: {num_transactions}")

Number of transactions: 119544


# Prepare dictionary

### Currency dictionary

In [5]:
# Define the index to currency mapping
currency_index = {
    0: 'BTC', 1: 'USD', 2: 'LTC', 3: 'ETH', 4: 'ETC', 5: 'USDT', 6: 'XMR', 7: 'BCH', 8: 'BTG', 9: 'B2X', 
    10: 'SBTC', 11: 'SGD', 12: 'CAD', 13: 'EUR', 14: 'NTD', 15: 'JPY', 16: 'GBP', 17: 'LCH', 18: 'CHF', 
    19: 'ZEC', 20: 'ADA', 21: 'MYR', 22: 'DAI', 23: 'XSGD', 24: 'XIDR'}

### Currency_pair dictionary

In [6]:
# API endpoint URL
url = 'https://api.coinut.com/spot'

# Send a GET request to the API
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Get the JSON data from the response
    data = response.json()
    
# Initialize an empty dictionary to store the values
pair_dic = {}

# Iterate over the data
for item in data:
    asset = item['asset']
    inst_id = item['inst_id']
    
    # Add the values to the dictionary with asset as the key
    pair_dic[inst_id] = asset


In [7]:
old_dic={}
# Open the JSON file
with open('oldAPI.json') as file:
    data2 = json.load(file)

# Iterate over the data
for item in data2:
    asset = item['asset']
    inst_id = item['inst_id']
    
    # Add the values to the dictionary with asset as the key
    old_dic[inst_id] = asset

In [8]:
# Merge the dictionaries
pair_dic.update(old_dic)

print (pair_dic)
# Initialize a new dictionary to store the split pairs
split_index_dic = {}

# For each pair in the original dictionary
for pair_id, pair in pair_dic.items():
    # Determine the split index based on the length of the pair string
    if len(pair) in [6, 8]:
        split_index = len(pair) // 2
    else: # len(pair) == 7
        split_index = 3
        # If the second half of the pair is not in the list of currencies, change the split index
        if pair[split_index:] not in currency_index.values():
            split_index = 4

    # Split the pair string at the split index
    first_currency = pair[:split_index]
    second_currency = pair[split_index:]

    # Add the split pair to the dictionary, joined by a comma
    split_index_dic[pair_id] = ', '.join([first_currency, second_currency])

print(split_index_dic)



{1: 'LTCBTC', 2: 'ETHBTC', 3: 'ETCBTC', 490569: 'ETHLTC', 490588: 'ETCLTC', 490589: 'BTCUSDT', 490590: 'LTCUSDT', 490591: 'ETHUSDT', 490592: 'ETCUSDT', 852382: 'XMRUSDT', 852383: 'XMRBTC', 852384: 'XMRLTC', 1111976: 'ZECUSDT', 1111977: 'ZECBTC', 1111978: 'ZECLTC', 1801843: 'BTCCAD', 1801844: 'LTCCAD', 1801845: 'ETHCAD', 1885998: 'ZECCAD', 2372505: 'BTCXSGD', 2372506: 'LTCXSGD', 2372507: 'ETHXSGD', 2372508: 'USDTXSGD', 2372510: 'DAIXSGD', 2372511: 'ETCXSGD', 2372512: 'BTCXIDR', 852220: 'USDTUSD', 852234: 'LTCUSD', 852379: 'BTCUSD', 852380: 'BTCSGD', 852381: 'LTCSGD', 899746: 'USDTSGD', 1063804: 'ETHSGD', 1885996: 'ZECUSD', 1885999: 'ETHUSD', 2372503: 'ETCSGD', 2372504: 'DAISGD', 2372509: 'XSGDSGD'}
{1: 'LTC, BTC', 2: 'ETH, BTC', 3: 'ETC, BTC', 490569: 'ETH, LTC', 490588: 'ETC, LTC', 490589: 'BTC, USDT', 490590: 'LTC, USDT', 490591: 'ETH, USDT', 490592: 'ETC, USDT', 852382: 'XMR, USDT', 852383: 'XMR, BTC', 852384: 'XMR, LTC', 1111976: 'ZEC, USDT', 1111977: 'ZEC, BTC', 1111978: 'ZEC, LTC'

### Daily_price dictionary

In [9]:
daily_price_dict={}

mysql_cursor.execute("SELECT * FROM price")
rows = mysql_cursor.fetchall()
columns = [column[0] for column in mysql_cursor.description]
 
for row in rows:
    row_dict = dict(zip(columns, row))
    date = row_dict['time'].strftime('%Y-%m-%d')
    daily_price_dict[date] = {currency: row_dict[currency] for currency in columns if currency != 'time' and currency != 'id'}

## Transaction Paires Analysis

In [11]:
# Initialize a dictionary to store the counts and amounts
transaction_details = {}
transaction_totals = {}  # Stores total transaction value in SGD for each pair

# Initialize transaction counter
transaction_counter = 0

# Iterate over the transactions
for transaction in data1:
    inst_id = transaction['order']['instId']
    pair = split_index_dic.get(inst_id, None)
    
    
    if pair:
        # Split the currency pair
        first_currency, second_currency = pair.split(', ')

        # Calculate the amounts
        first_currency_amount = float(transaction['fillQty'])
        second_currency_amount = float(transaction['fillPrice']) * float(transaction['fillQty'])

        # If the pair is not yet in the dictionary, add it with the count and amounts
        if pair not in transaction_details:
            transaction_details[pair] = {'Count': 1, first_currency: first_currency_amount, second_currency: second_currency_amount}
        else:
            # If the pair is already in the dictionary, increment the count and add the amounts
            transaction_details[pair]['Count'] += 1
            transaction_details[pair][first_currency] += first_currency_amount
            transaction_details[pair][second_currency] += second_currency_amount
            
       
       # Convert timestamp to datetime format
        timestamp = transaction['timestamp']
        dt = datetime.utcfromtimestamp(int(timestamp) / 1e6).strftime('%Y-%m-%d %H:%M:%S.%f')
        dt_date = dt.split()[0]  # get the date part
        
        
        # Initialize fill_qty and base_currency_price
        fill_qty = 0
        base_currency_price = 0
        sgd_price=0
        
        # Check if we have price data for this date
        daily_prices = daily_price_dict.get(dt_date, {})

        # Check if we have price data for this date
        if dt_date in daily_price_dict and first_currency in daily_price_dict[dt_date]:
            base_currency_price = daily_price_dict[dt_date][first_currency]
            fill_qty = first_currency_amount  # Using first_currency_amount instead of getting from transaction again
        
        # Check if we have sgd_price for this date, if not or if it's zero, use 0.74
        sgd_price = daily_prices.get('SGD', 0.74) if daily_prices.get('SGD', 0) != 0 else 0.74

        # Calculate the transaction value in SGD
        transaction_value_sgd = fill_qty *  (float(base_currency_price)/float(sgd_price))

        # Add the transaction value to the transaction document
        transaction['transaction_value_sgd'] = transaction_value_sgd
        
        # Increment transaction counter
        transaction_counter += 1
        
        # Update the total for the pair
        if pair in transaction_totals:
            transaction_totals[pair] += transaction_value_sgd
        else:
            transaction_totals[pair] = transaction_value_sgd


        


 # Print the transaction details
for pair, details in transaction_details.items():
   # Split the currency pair
    first_currency, second_currency = pair.split(', ')

        
    print(f'Pair: {pair}, Count: {details["Count"]}, {first_currency}: {details[first_currency]:.2f}, {second_currency}: {details[second_currency]:.2f}, Total SGD: {transaction_totals[pair]:.2f}')

# Print the transaction counter
print(f'Total transactions processed: {transaction_counter}')

Pair: LTC, USDT, Count: 63237, LTC: 23915.26, USDT: 1569842.06, Total SGD: 2175495.74
Pair: ETH, XSGD, Count: 1323, ETH: 331.84, XSGD: 859274.85, Total SGD: 857182.26
Pair: BTC, XSGD, Count: 2778, BTC: 36.08, XSGD: 1340458.01, Total SGD: 1423655.20
Pair: ZEC, LTC, Count: 1170, ZEC: 6.50, LTC: 7.89, Total SGD: 1020.36
Pair: ETH, LTC, Count: 782, ETH: 2.11, LTC: 48.66, Total SGD: 6751.81
Pair: ETC, LTC, Count: 11890, ETC: 283.93, LTC: 82.32, Total SGD: 8477.32
Pair: XMR, USDT, Count: 3375, XMR: 216.16, USDT: 41943.87, Total SGD: 57009.79
Pair: USDT, XSGD, Count: 4522, USDT: 3718059.37, XSGD: 5056328.39, Total SGD: 5057418.87
Pair: LTC, XSGD, Count: 779, LTC: 1787.02, XSGD: 203744.84, Total SGD: 203502.15
Pair: ETH, USDT, Count: 5122, ETH: 318.73, USDT: 865657.62, Total SGD: 1174732.68
Pair: ZEC, USDT, Count: 2111, ZEC: 283.12, USDT: 34812.82, Total SGD: 47599.03
Pair: ETC, USDT, Count: 3340, ETC: 3211.60, USDT: 81117.41, Total SGD: 110234.43
Pair: BTC, USDT, Count: 3844, BTC: 31.02, USDT

# Sort by Total SGD

In [12]:
# Create a list of dictionaries
transaction_list = []
for pair, details in transaction_details.items():
    # Split the currency pair
    first_currency, second_currency = pair.split(', ')

    transaction_list.append({
        'Pair': pair,
        'Count': details["Count"],
        first_currency: details[first_currency],
        second_currency: details[second_currency],
        'Total SGD': transaction_totals[pair]
    })

# Sort the list of dictionaries by 'Total SGD' in descending order
sorted_transaction_list = sorted(transaction_list, key=lambda x: x['Total SGD'], reverse=True)

# Print the sorted transaction list
for transaction in sorted_transaction_list:
   # Split the currency pair
    first_currency, second_currency = transaction['Pair'].split(', ')
    
    print(f"Pair: {transaction['Pair']}, Count: {transaction['Count']}, {first_currency}: {transaction[first_currency]:.2f}, {second_currency}: {transaction[second_currency]:.2f}, Total SGD: {transaction['Total SGD']:.2f}")

Pair: USDT, XSGD, Count: 4522, USDT: 3718059.37, XSGD: 5056328.39, Total SGD: 5057418.87
Pair: LTC, USDT, Count: 63237, LTC: 23915.26, USDT: 1569842.06, Total SGD: 2175495.74
Pair: BTC, XSGD, Count: 2778, BTC: 36.08, XSGD: 1340458.01, Total SGD: 1423655.20
Pair: ETH, USDT, Count: 5122, ETH: 318.73, USDT: 865657.62, Total SGD: 1174732.68
Pair: BTC, USDT, Count: 3844, BTC: 31.02, USDT: 746222.88, Total SGD: 1017923.94
Pair: ETH, XSGD, Count: 1323, ETH: 331.84, XSGD: 859274.85, Total SGD: 857182.26
Pair: ETC, XSGD, Count: 381, ETC: 6705.82, XSGD: 330100.90, Total SGD: 325773.99
Pair: LTC, XSGD, Count: 779, LTC: 1787.02, XSGD: 203744.84, Total SGD: 203502.15
Pair: BTC, XIDR, Count: 389, BTC: 6.74, XIDR: 1825824370.60, Total SGD: 154931.70
Pair: ETC, USDT, Count: 3340, ETC: 3211.60, USDT: 81117.41, Total SGD: 110234.43
Pair: LTC, BTC, Count: 1240, LTC: 738.19, BTC: 2.23, Total SGD: 94611.68
Pair: USDT, USD, Count: 8681, USDT: 66370.30, USD: 66387.16, Total SGD: 90291.95
Pair: XMR, USDT, Cou

# Sort by Total Count

In [13]:
# Sort the list of dictionaries by 'Total SGD' in descending order
sorted_transaction_list = sorted(transaction_list, key=lambda x: x['Count'], reverse=True)

# Print the sorted transaction list
for transaction in sorted_transaction_list:
   # Split the currency pair
    first_currency, second_currency = transaction['Pair'].split(', ')
    
    print(f"Pair: {transaction['Pair']}, Count: {transaction['Count']}, {first_currency}: {transaction[first_currency]:.2f}, {second_currency}: {transaction[second_currency]:.2f}, Total SGD: {transaction['Total SGD']:.2f}")

Pair: LTC, USDT, Count: 63237, LTC: 23915.26, USDT: 1569842.06, Total SGD: 2175495.74
Pair: ETC, LTC, Count: 11890, ETC: 283.93, LTC: 82.32, Total SGD: 8477.32
Pair: USDT, USD, Count: 8681, USDT: 66370.30, USD: 66387.16, Total SGD: 90291.95
Pair: ETH, USDT, Count: 5122, ETH: 318.73, USDT: 865657.62, Total SGD: 1174732.68
Pair: USDT, XSGD, Count: 4522, USDT: 3718059.37, XSGD: 5056328.39, Total SGD: 5057418.87
Pair: BTC, USDT, Count: 3844, BTC: 31.02, USDT: 746222.88, Total SGD: 1017923.94
Pair: LTC, USD, Count: 3386, LTC: 381.50, USD: 36782.93, Total SGD: 50352.03
Pair: XMR, USDT, Count: 3375, XMR: 216.16, USDT: 41943.87, Total SGD: 57009.79
Pair: ETC, USDT, Count: 3340, ETC: 3211.60, USDT: 81117.41, Total SGD: 110234.43
Pair: BTC, XSGD, Count: 2778, BTC: 36.08, XSGD: 1340458.01, Total SGD: 1423655.20
Pair: ZEC, USDT, Count: 2111, ZEC: 283.12, USDT: 34812.82, Total SGD: 47599.03
Pair: ETH, XSGD, Count: 1323, ETH: 331.84, XSGD: 859274.85, Total SGD: 857182.26
Pair: LTC, BTC, Count: 1240,